- Remove non alphanumeric characters for simple training

In [81]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [82]:
english_file = 'train.en' # replace this path with appropriate one
kannada_file = 'train.kn' # replace this path with appropriate one

# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                      'ँ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ',
                      'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ', 'ಌ', 'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ',
                      'ಕ', 'ಖ', 'ಗ', 'ಘ', 'ಙ',
                      'ಚ', 'ಛ', 'ಜ', 'ಝ', 'ಞ',
                      'ಟ', 'ಠ', 'ಡ', 'ಢ', 'ಣ',
                      'ತ', 'ಥ', 'ದ', 'ಧ', 'ನ',
                      'ಪ', 'ಫ', 'ಬ', 'ಭ', 'ಮ',
                      'ಯ', 'ರ', 'ಱ', 'ಲ', 'ಳ', 'ವ', 'ಶ', 'ಷ', 'ಸ', 'ಹ',
                      '಼', 'ಽ', 'ಾ', 'ಿ', 'ೀ', 'ು', 'ೂ', 'ೃ', 'ೄ', 'ೆ', 'ೇ', 'ೈ', 'ೊ', 'ೋ', 'ೌ', '್', 'ೕ', 'ೖ', 'ೞ', 'ೣ', 'ಂ', 'ಃ',
                      '೦', '೧', '೨', '೩', '೪', '೫', '೬', '೭', '೮', '೯', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [83]:
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)}
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [84]:
with open(english_file, encoding='utf-16-be') as file:
    english_sentences = file.readlines()
with open(kannada_file,encoding='utf-16-le') as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 10000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [85]:
english_sentences[:10]
print(english_sentences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [86]:
kannada_sentences[:10]
print(kannada_sentences)

['닠\ue087떲닠\ue0b0膳\ue020뢲닠\ue082뚲돠\ue08bꞲ닠ₕ닠\ue0b8足닠\ue0b5궲닠\ue0be떲닠\ue0a6떲닠\ue0b0膳ਮ\ue022蚲닠\ue0a6낲돠₆닠\ue0b8꒲돠\ue08d꾲\ue020릲돠\ue08a낲\ue020겲닠\ue082ꚲ돠₇닠\ue0ac낲돠\ue081꒲돠\ue08d꒲닠\ue0a6蚳\ue020躲닠\ue082ꚲ돠₁닠\ue0b9螳닠\ue0b3뾲닠₦닠\ue0b0뺲닠\ue0b9膳닠\ue0b2足\ue020鞲닠\ue0be芲닠\ue0a7뾲\u202c∢닠\ue0b8芳닠\ue0b0꒲돠₍닠\ue09cꢲ닠\ue0b0膳\ue020骲돠\ue080ꢲ닠\ue0beꚲ\ue020鲲닠\ue0a4蚳\ue020뢲돠\ue08dꪲ닠\ue0b0足닠\ue0a7蚳\ue020ꢲ닠\ue0a1蚳닠\ue0b8膳닠\ue0a4足닠\ue0a4뾲닠\ue0a6足닠\ue0a6뺲닠\ue0b0蚳ਢ닠\ue095뎲돠\ue08d뎲닠\ue0a4ꢲ닠\ue0b5뺲닠\ue097뾲닠\ue0a6足닠₦‸닠\ue0b2閲돠\ue08d랲\ue020낲돠⺂\ue00a螲닠\ue0a6낲\ue020겲닠\ue097足닠\ue097蚳\ue020ꢲ닠\ue0beꢲ돠₂닠\ue0b8뺲닠\ue095랲돠\ue08d龲돠₁닠\ue093ꚲ닠\ue0bfꚲ돠\ue08dꚲ돠\ue087ꢲ돠⺆\ue00a蚲닠\ue095蚳닠₯닠\ue0a4늲돠\ue086꾲\ue020릲닠\ue0a4足닠\ue0a4뾲닠₰닠\ue087낲닠\ue0bf뢲닠\ue0bf閲돠\ue08a芲닠\ue0a1뾲닠\ue0a6足닠₦닠\ue0ab讳닠\ue0a8足胢\ue08cꢲ\ue020겲돠\ue08d꾲닠\ue0be龲닠\ue0b0뾲\ue020뢲돠\ue08dꮲ돠\ue08b龲닠\ue097誳닠\ue082ꆲ돠₁닠\ue086閲돠₆닠\ue0ae莳닠\ue0a4ꪲ닠\ue09f足닠\ue09f뾲닠\ue0a6足닠\ue0a6뺲닠\ue0b3蚳\ue020躲닠\ue0a8足닠\ue0a8늲닠\ue0be鞲닠\ue0bfꚲ돠⺆\ue00a꺲닠\ue0beꢲ닠\ue0b5閲돠\ue081늲닠\ue0b5膳\ue020뢲돠\ue088꒲닠\ue0be

In [87]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in kannada_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 524288.0
97th percentile length English: 49807360.0


In [88]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 1
Number of valid sentences: 0


In [89]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [90]:
kannada_sentences[:3]

[]

In [91]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          kannada_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [92]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [93]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [94]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [95]:
len(dataset)

0

In [96]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [97]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

In [98]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [99]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [100]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == kannada_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_kannada[idx.item()]
            print(f"Kannada Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


## Inference

In [101]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_kannada[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [102]:
translation = translate("what should we do when the day starts?")
print(translation)


ೌ----ೌಯಯ--------------ರಯಯಯರರಫಫಫಫಫ೪ಜಜಜ೪೪೪----೪೪೪೪೪------ಜಜಜ೪೪೪%----೪-------------------------------ಫಫ---ಫಫಫಫಫ----------------------------------ಭಭಭರರರರರರರರರಭಭಭಭಭಭಭಭಭಭಭಭಭ-----ರರ----೪------ಯಯ-------------


In [103]:
translation = translate("i cannot stand this smell")
print(translation)


೪೪೪-++------------೫೫ಭಭ೪೪೪೪೪೪ಫಫಫಫ೪೪೪ಜ೪೪೪೪೪--೪೪೪೪೪೪----%ಜಜಜಜ೪೪೪%%%%೪೪೪೪--%%----------%%%-----------ಫಫಫಫಫ##ಫಫಫಫ##------------------------------ಭಭಭಭಭಭರರರರ-ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ-----೪೪೪೪೪೪೪-----ಯಯ-------------


In [104]:
translation = translate("noodles are the best")
print(translation)

೪೪೪--ೌ-------------೫ಭಭ೪೪೪೪೪೪ಫಫಫಫ೪೪೪೪೪೪೪೪೪--೪೪೪೪೪೪----%%ಜಜ೪೪೪೪%%%%೪೪೪೪--%%-----------%%-----------ಫಫಫಫಫ###ಫಫಫ###------------------------------ಭಭಭಭಭರರರ--ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ------೪೪೪೪೪೪೪--------------------


In [105]:
translation = translate("this is the best thing ever")
print(translation)


೪೪--++------------೫೫ಭಭ೪೪೪೪೪ರಫಫಫಫ೪೪ಜ೪ಜ೪೪೪೪--೪೪೪೪೪೪----%ಜಜಜಜ೪೪೪%%%ಜ೪೪೪೪--%%-----------%%-----------ಫಫಫಫಫಫ#ಫಫಫಫ##------------------------------ಭಭಭಭಭಭರರರರರಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ----ರ೪೪೪೪೪೪೪-----ಯಯ-----ಯ-------


In [106]:
translation = translate("i am here")
print(translation)


೪೪೪೪೪೪೪))))-----೪೫೫ಭಭ೪೪೪೪೪೪೪೪ಫಫ೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೫ಜ೪೪೪೪೪೪೪%೪೪೪೪೪೪%%%%---------%%%---ಭಭಭ-----ಫಫಫ############೪-----------೪೪೪ಣಣ-----ಭಭಭಭಭಭಭಭಭಭಭಭಭರಣಣಣಭಭಭಭಭಭಭಭ೪೪ಭಭಭಭಭಭಭ---೪೪೪೪೪೪೪೪೪೪೪))))೪)೪)ుಶಶಶ--ుుుಭ


In [107]:
translation = translate("click this")
print(translation)


೪೪೪೪೪೪೪&))------೪೫೫ಭಭ೪೪೪೪೪೪೪೪ಫಫ೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪%೪ಜ೪೪೪೪೪೪%%೪೪೪೪೪೪%%%%---------%%%---ಭಭಭ-----ಫಫಫಫ#####ಫ#####-------------೪೪ಣಣ-----ಭಭಭಭಭಭಭಭಭಭಭಭಭರರಣಣಭಭಭಭಭಭಭಭ೪೪ಭಭಭಭಭಭಭ---೪೪೪೪೪೪೪೪೪೪೪)))ಯ)೪))ుಶಯಶ---ుుಭ


In [108]:
translation = translate("where is the mall?")
print(translation)

೪೪೪--ೌ-------------೫ಭ೪೪೪೪೪೪೪ಫಫಫಫ೪೪೪೪೪೪೪೪೪--೪೪೪೪೪೪೪---%%ಜಜ೪೪೪೪%%%%೪೪೪೪--%%-----------%%-----------ಫಫಫಫಫ###ಫಫಫ###------------------------------ಭಭಭಭಭರರರ--ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ-----೪೪೪೪೪೪೪೪೪--------ుుు--------


In [109]:
translation = translate("what should we do?")
print(translation)

೪೪೪--ೌ-------------೫ಭ೪೪೪೪೪೪೪ಫಫಫಫ೪೪೪೪೪೪೪೪೪--೪೪೪೪೪೪೪---%%ಜಜ೪೪೪೪%%%%೪೪೪೪--%%%---------%%%-----------ಫಫಫಫಫ###ಫಫಫ###------------------------------ಭಭಭಭಭರರರ--ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ-----೪೪೪೪೪೪೪೪೪--------ుుుಯ-------


In [110]:
translation = translate("today, what should we do")
print(translation)

೪೪---ೌಉ೫----------೫೫ಭಭ೪೪೪೪೪೪ಫಫಫಫ೪೪೪ಜ೪೪೪೪೪--೪೪೪೪೪೪----%%ಜಜಜ೪೪೪%%%%೪೪೪೪--%%----------%%%-----------ಫಫಫಫಫ##ಫಫಫಫ##------------------------------ಭಭಭಭಭಭರರರರ-ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ-------೪೪೪೪೪-----ಯಯ-------------


In [111]:
translation = translate("why did they activate?")
print(translation)


೪೪---ೌ------------೫೫ಭಭ೪೪೪೪೪೪ಫಫಫಫ೪೪೪೪೪೪೪೪೪--೪೪೪೪೪೪----%%ಜಜ೪೪೪೪%%%%೪೪೪೪--%%----------%%%-----------ಫಫಫಫಫ##ಫಫಫಫ##-------------------------------ಭಭಭಭಭರರರ--ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ---------೪೪೪೪--------------------


In [112]:
translation = translate("why did they do this?")
print(translation)


೪೪---ೌ-------------೫ಭಭ೪೪೪೪೪೪ಫಫಫಫ೪೪೪೪೪೪೪೪೪--೪೪೪೪೪೪----%%ಜಜ೪೪೪೪%%%%೪೪೪೪--%%-----------%%-----------ಫಫಫಫಫ##ಫಫಫಫ##-------------------------------ಭಭಭಭಭರರರ--ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ---------೪೪೪೪--------------------


In [113]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

೪೪೪೪೪೪೪&))------೪೫೫ಭಭ೪೪೪೪೪೪೪೪ಫಫ೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪೪%೪ಜ೪೪೪೪೪೪%%೪೪೪೪೪೪%%%%---------%%%---ಭಭಭ-----ಫಫಫಫ#####ಫ#####-------------೪೪ಣಣ-----ಭಭಭಭಭಭಭಭಭಭಭಭಭರರಣಣಭಭಭಭಭಭಭಭ೪೪ಭಭಭಭಭಭಭ---೪೪೪೪೪೪೪೪೪೪೪))))೪)೪)ుಶಯಶ---ుుಭ


In [114]:
translation = translate("whats the word on the street?")
print(translation)

ೌ೪--ೌಯಯಯ----------೫೫ಭಭ೪೪೪೪೪ರಫಫಫಫ೪೪ಜಜಜ೪೪೪೪--೪೪೪೪೪೪-----ಜಜಜಜ೪೪೪%%%ಜ೪೪೪----------------%%-----------ಫಫಫಫಫಫ#ಫಫಫಫಫ#------------------------------ಭಭಭಭಭಭರರರರರಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ----ರರ೪೪೪೪೪೪-----ಯಯಯ----ಯಯ------
